# === USER CONFIGURATION ===


In [ ]:
# # === USER CONFIGURATION ===
# /!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\
# /!\/!\/!\ CAREFUL, please read the README_Notebooks.md file before
# /!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\

WORKDIR = "PATH/TO/WORKDIR"  # Working directory for sen2vm processing
# /!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\
# /!\/!\/!\ CAREFUL, the structure of the WORKDIR shall respect the one described in README_Notebooks.md
# /!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\

# Path to downloaded product
PATH_L1B_DATA = "PATH/TO/L1B/PRODUCT"
# /!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\
# /!\/!\/!\ CAREFUL, for now PATH_L1B_DATA shall be inside WORKID/DATA and respect the one described in README_Notebooks.md
# /!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\

# Output folder chosen by the user
INVERSE_OUTPUT_FOLDER = "PATH/TO/OUTPUT/FOLDER"  # Used only if GRID_MODE = "inverse"


# === SEN2VM OPTIONS ===

GRID_MODE = "direct"  # direct or inverse
# /!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\
# /!\/!\/!\ CAREFUL, for now only direct shall be used with a full product (no missing granules)
# /!\/!\/!\/!\/!\/!\/!\/!\/!\/!\/!\

UTM_EPSG = 23037 # UTM zone EPSG code for the ROI
LOCATION =  {
    "ul_x": -185299,
    "ul_y": 3363178,
    "lr_x": -17915,
    "lr_y": 3352896
}

STEPS = {
    "10m_bands": 10,
    "20m_bands": 20,
    "60m_bands": 60
}

# === GDAL ORTHO OPTIONS ===

DELETE_TMP_FILES = True  # Delete temporary files created during orthorectification

ORTHO_SETTINGS = {
    "keep_bands": ["B01" , "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B10", "B11", "B12"], # list of bands to keep for orthorectification
}
 


# GIPP database download (Optionnal)

In [ ]:
# === GIPPs ===
# This step is optionnal if the Database was already downloaded or if you want to use your own GIPP
# Please note that this current notebook will search for a subfolder with mission S2[A/B/C] inside the GIPP folder

import os
import shutil
import tarfile
import re
from datetime import datetime


gipp_dir = os.path.join(WORKDIR, "DATA")     
os.makedirs(gipp_dir, exist_ok=True)             

# =====================================================================
# 1) CLONE sen2vm-gipp-database
# =====================================================================

gipp_repo_name = "sen2vm-gipp-database"
gipp_repo_path = os.path.join(gipp_dir, gipp_repo_name)

# Delete repository if exists
if os.path.exists(gipp_repo_path):
    print(f"Removing existing repository: {gipp_repo_path}")
    shutil.rmtree(gipp_repo_path)

# Clone repository fresh
print("Cloning sen2vm-gipp-database...")
!git clone https://github.com/sen2vm/sen2vm-gipp-database.git {gipp_repo_path}
print("Clone complete.\n")



print("GIPP processing finished successfully.")


# IERS Download (optionnal)

In [ ]:
# === DOWNLOAD IERS ===

import os
import re
import requests
from datetime import datetime

DATA_DIR = os.path.join(WORKDIR, "DATA")

if not os.path.exists(DATA_DIR):
    raise RuntimeError(f"DATA directory not found: {DATA_DIR}")

print("Bulletin output directory:", DATA_DIR)

# =====================================================================
# 0. REMOVE EXISTING IERS BULLETINS
# =====================================================================

for f in os.listdir(DATA_DIR):
    if f.startswith("bulletina-") and f.endswith(".txt"):
        os.remove(os.path.join(DATA_DIR, f))
        print("Removed old bulletin A:", f)

    if f.startswith("bulletinb-") and f.endswith(".txt"):
        os.remove(os.path.join(DATA_DIR, f))
        print("Removed old bulletin B:", f)

print("Cleanup of old bulletins complete.\n")

# =====================================================================
# 2. EXTRACT PRODUCT DATE (FROM DATASTRIP)
# =====================================================================

datastrip_dir = os.path.join(PATH_L1B_DATA, "DATASTRIP")

if not os.path.isdir(datastrip_dir):
    raise RuntimeError(f"DATASTRIP directory not found: {datastrip_dir}")

datastrip_entries = os.listdir(datastrip_dir)
if not datastrip_entries:
    raise RuntimeError(f"No DATASTRIP found in: {datastrip_dir}")

# Take the first DATASTRIP product
datastrip_name = datastrip_entries[0]

match = re.search(r"_S(\d{8})T\d{6}_", datastrip_name)
if not match:
    raise RuntimeError("Could not extract product date from DATASTRIP name.")

product_date_str = match.group(1)

year = int(product_date_str[:4])
month = int(product_date_str[4:6])
day = int(product_date_str[6:8])

product_date = datetime(year, month, day)

print("Product date extracted from DATASTRIP:", product_date.date(), "\n")

# =====================================================================
# Roman conversion 
# =====================================================================

def int_to_roman(n):
    vals = [
        (1000, 'm'), (900, 'cm'), (500, 'd'), (400, 'cd'),
        (100, 'c'), (90, 'xc'), (50, 'l'), (40, 'xl'),
        (10, 'x'), (9, 'ix'), (5, 'v'), (4, 'iv'), (1, 'i')
    ]
    res = ""
    for v, s in vals:
        while n >= v:
            res += s
            n -= v
    return res


# =====================================================================
# Bulletin A
# =====================================================================


roman_year = int_to_roman(year - 1987)
doy = product_date.timetuple().tm_yday
index = (doy - 1) // 7 + 1

print(f"Bulletin A Roman year: {roman_year}")
print(f"Initial weekly index: {index}\n")

found = False

while index > 0:
    index_str = f"{index:03d}"
    url = f"https://datacenter.iers.org/data/6/bulletina-{roman_year}-{index_str}.txt"
    print("Trying bulletin:", url)

    response = requests.get(url)

    if response.status_code == 200:
        print("Bulletin found:", index_str)
        dest_file = os.path.join(DATA_DIR, f"bulletina-{roman_year}-{index_str}.txt")
        found = True
        break

    index -= 1

if not found:
    raise RuntimeError("No Bulletin A available for current or previous weeks.")

with open(dest_file, "wb") as f:
    f.write(response.content)

print("Downloaded:", dest_file)


# Sen2VM configuration

In [ ]:
# === GENERATE CONFIG.JSON  ===

import os
import json
import re
import shutil
from numpy import double


USERCONF_DIR = os.path.join(WORKDIR, "UserConf")
os.makedirs(USERCONF_DIR, exist_ok=True)

print("UserConf directory:", USERCONF_DIR)

GEOID_DIR = os.path.join(WORKDIR, "DATA", "GEOID")
os.makedirs(GEOID_DIR, exist_ok=True)

print("Geoid directory:", GEOID_DIR)

# =====================================================
# 1. Extract mission from DATASTRIP
# =====================================================

datastrip_dir = os.path.join(PATH_L1B_DATA, "DATASTRIP")

if not os.path.isdir(datastrip_dir):
    raise RuntimeError(f"DATASTRIP directory not found: {datastrip_dir}")

datastrip_entries = os.listdir(datastrip_dir)
if not datastrip_entries:
    raise RuntimeError(f"No DATASTRIP found in: {datastrip_dir}")

datastrip_name = datastrip_entries[0]

match = re.match(r"(S2[A-C])_OPER_", datastrip_name)
if not match:
    raise RuntimeError("Cannot extract mission (S2A/S2B/S2C) from DATASTRIP name.")

mission = match.group(1)


# =====================================================
# 2. Docker paths inside /workspace
# =====================================================

safe_name = os.path.basename(PATH_L1B_DATA)

docker_l1b  = f"/workspace/DATA/{safe_name}"
docker_dem  = "/workspace/DATA/DEM"
docker_gipp = f"/workspace/DATA/sen2vm-gipp-database/{mission}"

# =====================================================
# 3. Geoid management
# =====================================================

# Notebook location (NOT relative to CWD)
notebook_dir = os.getcwd()

# Relative path to DEM_GEOID from notebook
internal_geoid_dir = os.path.abspath(os.path.join(
    notebook_dir,
    "..", "..", "src", "test", "resources", "DEM_GEOID"
))

# If WORKDIR/DATA/GEOID is empty -> copy internal files
if len(os.listdir(GEOID_DIR)) == 0:
    print("GEOID directory is empty -> copying default geoid files...")
    for f in os.listdir(internal_geoid_dir):
        src = os.path.join(internal_geoid_dir, f)
        dst = os.path.join(GEOID_DIR, f)
        shutil.copy(src, dst)

# Detect .gtx inside GEOID_DIR
geoid_files = [f for f in os.listdir(GEOID_DIR) if f.lower().endswith(".gtx")]
if len(geoid_files) == 0:
    raise RuntimeError("No .gtx geoid file found in WORKDIR/DATA/GEOID.")

docker_geoid = f"/workspace/DATA/GEOID/{geoid_files[0]}"

# =====================================================
# 4. Locate IERS bulletin on host
# =====================================================

DATA_DIR = os.path.join(WORKDIR, "DATA")
iers_host = None

for f in os.listdir(DATA_DIR):
    if f.startswith("bulletin"):
        iers_host = os.path.join(DATA_DIR, f)
        break

if iers_host is None:
    raise RuntimeError("IERS bulletin not found inside WORKDIR/DATA directory.")

docker_iers = "/workspace/DATA/" + os.path.basename(iers_host)

# =====================================================
# 5. Build config dictionary
# =====================================================

config = {
    "l1b_product": docker_l1b,
    "gipp_folder": docker_gipp,
    "auto_gipp_selection": True,
    "grids_overwriting": True,
    "dem": docker_dem,
    "geoid": docker_geoid,
    "iers": docker_iers,
    "operation": GRID_MODE,
    "deactivate_available_refining": False,
    "steps": {
        "10m_bands": STEPS["10m_bands"],
        "20m_bands": STEPS["20m_bands"],
        "60m_bands": STEPS["60m_bands"]
    },
    "export_alt": True
}

# Add inverse block only if needed
if GRID_MODE == "inverse":
    config["inverse_location_additional_info"] = {
        "ul_x": double(LOCATION["ul_x"]),
        "ul_y": double(LOCATION["ul_y"]),
        "lr_x": double(LOCATION["lr_x"]),
        "lr_y": double(LOCATION["lr_y"]),
        "referential": f"EPSG:{UTM_EPSG}",
        "output_folder": "/workspace/DATA/Output"
    }

# =====================================================
# Save config.json
# =====================================================

config_path = os.path.join(USERCONF_DIR, "config.json")

with open(config_path, "w") as f:
    json.dump(config, f, indent=4)

print("Configuration file generated:")
print(config_path)


In [ ]:
# === GENERATE PARAMS.JSON ===

import os
import json
import re

USERCONF_DIR = os.path.join(WORKDIR, "UserConf")
os.makedirs(USERCONF_DIR, exist_ok=True)

print("UserConf directory:", USERCONF_DIR)

# =====================================================
# Locate GRANULE folders
# =====================================================
GR_TARGET_DIR = os.path.join(PATH_L1B_DATA, "GRANULE")

if not os.path.exists(GR_TARGET_DIR):
    raise RuntimeError("GRANULE directory not found inside L1B SAFE.")

granule_folders = [
    os.path.join(GR_TARGET_DIR, d)
    for d in os.listdir(GR_TARGET_DIR)
    if os.path.isdir(os.path.join(GR_TARGET_DIR, d))
]

print("Found", len(granule_folders), "granule folders.")

# =====================================================
# Extract detectors and bands from JP2
# =====================================================
detectors = set()
bands = set()

pattern = r"_D(\d+)_B(\d{1,2}[A]?)\.jp2$"

for granule in granule_folders:
    img_data_dir = os.path.join(granule, "IMG_DATA")

    if not os.path.isdir(img_data_dir):
        continue

    for fname in os.listdir(img_data_dir):
        match = re.search(pattern, fname)
        if match:
            detectors.add(match.group(1))
            bands.add(f"B{match.group(2)}")

detectors = sorted(detectors)
bands = sorted(bands)

print("Detected detectors:", detectors)
print("Detected bands:", bands)

# =====================================================
# Write params.json
# =====================================================
params = {
    "detectors": detectors,
    "bands": bands
}

params_path = os.path.join(USERCONF_DIR, "params.json")

with open(params_path, "w") as f:
    json.dump(params, f, indent=4)

print("params.json written to:", params_path)


# Sen2VM run

In [ ]:
# === RUN SEN2VM (Docker: BUILD + RUN + CLEAN) ===

import os
import subprocess


dockerfile_dir = os.path.abspath(os.path.join(
    notebook_dir,
    "..", ".."
))

config_inside = "/workspace/UserConf/config.json"
params_inside = "/workspace/UserConf/params.json"

# =====================================================
# 1. BUILD DOCKER IMAGE
# =====================================================

print(f"Building Docker image 'sen2vm' from: {dockerfile_dir}")

cmd_build = [
    "docker", "build",
    "-t", "sen2vm",
    dockerfile_dir
]

print("Command:", " ".join(cmd_build), "\n")
subprocess.run(cmd_build, check=True)
print("Docker image built successfully.\n")

# =====================================================
# 2. RUN SEN2VM CONTAINER
# =====================================================

cmd_run = [
    "docker", "run",
    "--rm",
    "-v", f"{WORKDIR}:/workspace",  
    "sen2vm",
    "-c", config_inside,
    "-p", params_inside
]

print("Running Docker container...\n")
print("Command:", " ".join(cmd_run), "\n")

subprocess.run(cmd_run, check=True)

print("\nDocker execution complete.\n")

# =====================================================
# 3. REMOVE DOCKER IMAGE
# =====================================================

print("Removing Docker image 'sen2vm'...")

subprocess.run(["docker", "rmi", "-f", "sen2vm"], check=True)

print("Docker image removed.\n")

# TEST IF THE PRODUCT IS COMPLET


In [ ]:
import glob
import os
import re
import shutil
import sys
import xml.etree.ElementTree as ET
from collections import defaultdict
from typing import Set, List, Optional


# =====================================================
# Common helpers
# =====================================================
def _abspath(path: str) -> str:
    return os.path.abspath(os.path.expanduser(path))


def print_list(title: str, items: List[str], limit: Optional[int] = None) -> None:
    print(title)
    if not items:
        print("  (none)")
        return

    if limit is not None and limit > 0 and len(items) > limit:
        for it in items[:limit]:
            print("  -", it)
        print(f"  ... ({len(items) - limit} more)")
        return

    for it in items:
        print("  -", it)


def find_datastrip_xml(safe_path: str) -> str:
    patterns = [
        os.path.join(safe_path, "DATASTRIP", "S2*", "S2*_MTD_L1B_DS_*.xml"),
        os.path.join(safe_path, "DATASTRIP", "S2*", "MTD_L1B_DS_*.xml"),
    ]
    xmls: List[str] = []
    for p in patterns:
        xmls.extend(glob.glob(p))
    xmls = sorted(set(xmls))
    if not xmls:
        raise FileNotFoundError(f"No DATASTRIP MTD XML found under: {safe_path}/DATASTRIP")
    if len(xmls) > 1:
        print("WARNING: Multiple DATASTRIP XML found; using the first one:", file=sys.stderr)
        for x in xmls:
            print("  -", x, file=sys.stderr)
    return xmls[0]


def list_granule_dirs(safe_path: str) -> Set[str]:
    granule_root = os.path.join(safe_path, "GRANULE")
    if not os.path.isdir(granule_root):
        raise FileNotFoundError(f"Missing GRANULE directory: {granule_root}")
    return {e for e in os.listdir(granule_root) if os.path.isdir(os.path.join(granule_root, e))}


def extract_expected_granules_from_xml(xml_path: str) -> Set[str]:
    expected: Set[str] = set()

    tree = ET.parse(xml_path)
    root = tree.getroot()

    for elem in root.iter():
        if elem.text:
            text = elem.text.strip()
            if text.startswith("S2") and "_GR_" in text:
                expected.add(text)

        for val in elem.attrib.values():
            if val.startswith("S2") and "_GR_" in val:
                expected.add(val)

    if not expected:
        print("WARNING: No granule identifiers could be extracted from XML.\n", file=sys.stderr)

    return expected


def check_img_data_jp2(safe_path: str, granules: Set[str]) -> List[str]:
    problems: List[str] = []
    for g in sorted(granules):
        img_dir = os.path.join(safe_path, "GRANULE", g, "IMG_DATA")
        if not os.path.isdir(img_dir):
            problems.append(f"{g} -> IMG_DATA directory missing")
            continue
        if not glob.glob(os.path.join(img_dir, "*.jp2")):
            problems.append(f"{g} -> no *.jp2 found in IMG_DATA")
    return problems


# =====================================================
# Completeness check
# =====================================================
def is_complete(
    safe_path: str,
    check_img: bool = False,
    list_limit: int = 50,
) -> bool:
    safe_path = _abspath(safe_path)

    if not os.path.isdir(safe_path):
        print(f"ERROR: Not a directory: {safe_path}", file=sys.stderr)
        return False

    xml_path = find_datastrip_xml(safe_path)
    actual = list_granule_dirs(safe_path)
    expected = extract_expected_granules_from_xml(xml_path)

    present_in_both = sorted(actual & expected) if expected else []
    missing = sorted(expected - actual) if expected else []
    extra = sorted(actual - expected) if expected else []

    print("SAFE:", safe_path)
    print("DATASTRIP XML:", xml_path)
    print("Actual GRANULE count:", len(actual))
    print("Expected GRANULE count (from XML):", len(expected))
    print("Present in BOTH (disk ∩ XML) count:", len(present_in_both))
    print("Missing (XML - disk) count:", len(missing))
    print("Extra (disk - XML) count:", len(extra))
    print("")

    if expected:
        print_list("PRESENT IN BOTH (referenced by XML AND present on disk):", present_in_both, limit=list_limit)
        print("")
        print_list("MISSING GRANULES (referenced by XML but NOT present on disk):", missing, limit=list_limit)
        print("")
        print_list("EXTRA GRANULES (present on disk but NOT referenced by XML):", extra, limit=list_limit)
        print("")
    else:
        print("Skipping XML vs disk comparison (no expected granules extracted).")
        print("")

    img_problems: List[str] = []
    if check_img:
        to_check = set(present_in_both) if expected else actual
        img_problems = check_img_data_jp2(safe_path, to_check)

        if img_problems:
            print("IMG_DATA CHECK PROBLEMS:")
            for p in img_problems[:list_limit]:
                print("  -", p)
            if len(img_problems) > list_limit:
                print(f"  ... ({len(img_problems) - list_limit} more)")
            print("")
        else:
            print("IMG_DATA CHECK: OK")
            print("")

    return (len(missing) == 0) and (len(extra) == 0) and (len(img_problems) == 0)


# =====================================================
# TMP workdir copy only if incomplete product
# =====================================================
def ensure_workdir_tmp_and_copy_product(
    workdir: str,
    path_l1b_data: str,
    suffix: str = "_tmp",
) -> tuple[str, str]:
    workdir = _abspath(workdir)
    path_l1b_data = _abspath(path_l1b_data)

    workdir_parent = os.path.dirname(workdir)
    workdir_base = os.path.basename(workdir.rstrip(os.sep))
    workdir_tmp = os.path.join(workdir_parent, f"{workdir_base}{suffix}")

    if not path_l1b_data.startswith(workdir + os.sep):
        raise ValueError(
            "PATH_L1B_DATA must be located inside WORKDIR.\n"
            f"WORKDIR      : {workdir}\n"
            f"PATH_L1B_DATA : {path_l1b_data}"
        )

    rel_product_path = os.path.relpath(path_l1b_data, start=workdir)
    path_l1b_data_tmp = os.path.join(workdir_tmp, rel_product_path)

    os.makedirs(workdir_tmp, exist_ok=True)
    os.makedirs(os.path.dirname(path_l1b_data_tmp), exist_ok=True)

    if not os.path.exists(path_l1b_data_tmp):
        print(f"Copying WORKDIR -> WORKDIR_TMP:\n  {workdir}\n  -> {workdir_tmp}")
        shutil.copytree(workdir, workdir_tmp, dirs_exist_ok=True)
    else:
        print(f"WORKDIR_TMP already exists, not copying again:\n  {workdir_tmp}")

    return workdir_tmp, path_l1b_data_tmp


# =====================================================
# XML trim only if incomplete product
# =====================================================
def _extract_granule_ids_from_elem(elem: ET.Element) -> Set[str]:
    ids: Set[str] = set()
    if elem.text:
        t = elem.text.strip()
        if t.startswith("S2") and "_GR_" in t:
            ids.add(t)
    for v in elem.attrib.values():
        if v.startswith("S2") and "_GR_" in v:
            ids.add(v)
    return ids


def _extract_granule_ids_from_subtree(elem: ET.Element) -> Set[str]:
    ids: Set[str] = set()
    for e in elem.iter():
        ids |= _extract_granule_ids_from_elem(e)
    return ids


def _build_parent_map(root: ET.Element) -> dict[int, ET.Element]:
    parent_map: dict[int, ET.Element] = {}
    for parent in root.iter():
        for child in list(parent):
            parent_map[id(child)] = parent
    return parent_map


def trim_datastrip_xml_to_existing_granules(
    safe_path: str,
    xml_path: str,
) -> str:
    safe_path = _abspath(safe_path)
    xml_path = _abspath(xml_path)

    if not os.path.isdir(safe_path):
        raise FileNotFoundError(f"SAFE directory not found: {safe_path}")
    if not os.path.isfile(xml_path):
        raise FileNotFoundError(f"DATASTRIP XML not found: {xml_path}")

    actual_granules = list_granule_dirs(safe_path)

    tree = ET.parse(xml_path)
    root = tree.getroot()

    expected_granules = _extract_granule_ids_from_subtree(root)
    keep = actual_granules & expected_granules
    drop = expected_granules - keep

    print("TRIM DATASTRIP XML")
    print("SAFE:", safe_path)
    print("XML :", xml_path)
    print("Actual GRANULE count:", len(actual_granules))
    print("Expected GRANULE count (from XML):", len(expected_granules))
    print("KEEP (disk & XML) count:", len(keep))
    print("DROP (XML - KEEP) count:", len(drop))

    if not keep:
        raise RuntimeError("KEEP set is empty. Nothing to keep; aborting.")

    parent_map = _build_parent_map(root)
    candidates = list(root.iter())[::-1]

    removed = 0
    for elem in candidates:
        if elem is root:
            continue

        ids_in_elem = _extract_granule_ids_from_subtree(elem)
        if not ids_in_elem:
            continue
        if ids_in_elem & keep:
            continue
        if ids_in_elem.issubset(drop):
            parent = parent_map.get(id(elem))
            if parent is None:
                continue
            try:
                parent.remove(elem)
                removed += 1
            except ValueError:
                pass

    # Keep SAME output behavior as your original: overwrite input XML path
    tree.write(xml_path, encoding="utf-8", xml_declaration=True)

    print("Removed XML elements:", removed)
    print("Output written:", xml_path)
    print("")

    return xml_path


# =====================================================
# Position fix only if incomplete product
# =====================================================
DETECTOR_RE = re.compile(r"_D(\d{2})_")


def _localname(tag: str) -> str:
    return tag.split("}", 1)[1] if "}" in tag else tag


def _find_child(elem: ET.Element, child_localname: str) -> Optional[ET.Element]:
    for ch in list(elem):
        if _localname(ch.tag) == child_localname:
            return ch
    return None


def fix_datastrip_granule_positions(xml_path: str) -> str:
    xml_path = _abspath(xml_path)
    if not os.path.isfile(xml_path):
        raise FileNotFoundError(f"XML not found: {xml_path}")

    tree = ET.parse(xml_path)
    root = tree.getroot()

    granules = [e for e in root.iter() if _localname(e.tag) == "Granule"]
    if not granules:
        raise RuntimeError("No <Granule> elements found in this XML.")

    by_det: dict[str, List[tuple[str, int, ET.Element]]] = defaultdict(list)

    for g in granules:
        gid = g.attrib.get("granuleId", "")
        m = DETECTOR_RE.search(gid)
        det = f"D{m.group(1)}" if m else "UNKNOWN"

        pos_elem = _find_child(g, "POSITION")
        if pos_elem is None:
            continue
        try:
            pos_val = int((pos_elem.text or "").strip())
        except ValueError:
            continue

        by_det[det].append((gid, pos_val, g))

    if not by_det:
        raise RuntimeError("No granules with <POSITION> found.")

    print(f"Found {len(granules)} <Granule> elements")
    print(f"Detectors found: {', '.join(sorted(by_det.keys()))}")
    print("")

    for det in sorted(by_det.keys()):
        items = by_det[det]
        min_pos = min(p for _, p, _ in items)
        shift = min_pos - 1
        print(f"{det}: min POSITION={min_pos} -> shift={shift}")

        for gid, pos_val, g in sorted(items, key=lambda x: x[1]):
            pos_elem = _find_child(g, "POSITION")
            if pos_elem is None:
                continue
            new_pos = pos_val - shift
            if new_pos < 1:
                print(
                    f"WARNING: {det} would produce POSITION<1 for {gid} (old={pos_val}, new={new_pos})",
                    file=sys.stderr,
                )
            pos_elem.text = str(new_pos)

        print("  Example after fix (sorted by old POSITION):")
        for gid, pos_val, _ in sorted(items, key=lambda x: x[1])[:5]:
            print(f"    - {gid} : {pos_val} -> {pos_val - shift}")
        print("")

    # Keep SAME output behavior as your original: overwrite input XML path
    tree.write(xml_path, encoding="utf-8", xml_declaration=True)

    print(f"Written: {xml_path}")
    print("")

    return xml_path


# =====================================================
# Main
# =====================================================
product_is_complete = is_complete(PATH_L1B_DATA, check_img=True, list_limit=200)

if product_is_complete:
    print("L1B product is COMPLETE.")
else:
    print("L1B product is INCOMPLETE. Creating a tmp copy of WORKDIR...")

    WORKDIR_TMP, PATH_L1B_DATA_TMP = ensure_workdir_tmp_and_copy_product(
        workdir=WORKDIR,
        path_l1b_data=PATH_L1B_DATA,
        suffix="_tmp",
    )
    print("WORKDIR_TMP:", WORKDIR_TMP)
    print("PATH_L1B_DATA_TMP:", PATH_L1B_DATA_TMP)

    datastrip_xml_tmp = find_datastrip_xml(PATH_L1B_DATA_TMP)

    trimmed_xml_tmp = trim_datastrip_xml_to_existing_granules(
        safe_path=PATH_L1B_DATA_TMP,
        xml_path=datastrip_xml_tmp,
    )
    print("Trimmed XML path:", trimmed_xml_tmp)

    posfixed_xml_tmp = fix_datastrip_granule_positions(trimmed_xml_tmp)
    print("Position-fixed XML path:", posfixed_xml_tmp)


# Generate Orthorectification images

In [ ]:
import os
import subprocess
import glob


# =====================================================
# Choose inputs: normal vs _tmp (and keep outputs in WORKDIR)
# =====================================================
WORKDIR_ORIG = WORKDIR
PATH_L1B_DATA_ORIG = PATH_L1B_DATA

# Detect a usable TMP context (created by the previous cell)
use_tmp = (
    "WORKDIR_TMP" in globals()
    and "PATH_L1B_DATA_TMP" in globals()
    and isinstance(globals().get("WORKDIR_TMP"), str)
    and isinstance(globals().get("PATH_L1B_DATA_TMP"), str)
    and os.path.isdir(globals().get("WORKDIR_TMP"))
    and os.path.isdir(globals().get("PATH_L1B_DATA_TMP"))
)

# Switch inputs if tmp exists, but keep outputs in the original WORKDIR
WORKDIR_INPUT = globals().get("WORKDIR_TMP") if use_tmp else WORKDIR_ORIG
PATH_L1B_DATA_INPUT = globals().get("PATH_L1B_DATA_TMP") if use_tmp else PATH_L1B_DATA_ORIG

print("use_tmp:", use_tmp)
print("INPUT WORKDIR:", WORKDIR_INPUT)
print("INPUT PATH_L1B_DATA:", PATH_L1B_DATA_INPUT)
print("OUTPUT WORKDIR:", WORKDIR_ORIG)


# =====================================================
# Locate product name
# =====================================================
product = os.path.basename(os.path.normpath(PATH_L1B_DATA_INPUT))

# =====================================================
# Locate XML
# =====================================================
xml_list = glob.glob(
    os.path.join(
        PATH_L1B_DATA_INPUT,
        "DATASTRIP",
        "S2*",
        "S2*_MTD_L1B_DS_*.xml"
    )
)

if len(xml_list) == 0:
    raise RuntimeError("No DATASTRIP MTD XML found")

xml_path = xml_list[0]
xml_name = os.path.basename(xml_path)

# XML path inside docker (relative, after cd)
xml_docker = f"./{xml_name}"

print("Using XML:", xml_path)

# =====================================================
# Locate VRTs
# =====================================================
vrt_list = glob.glob(
    os.path.join(PATH_L1B_DATA_INPUT, "DATASTRIP", "S2*", "GEO_DATA", "*.vrt")
)

if not vrt_list:
    raise RuntimeError("No VRT files found in GEO_DATA")

vrt_names = [os.path.splitext(os.path.basename(v))[0] for v in vrt_list]

print("Found VRTs:", vrt_names)

# =====================================================
# Output directories
# =====================================================
OUTDIR = os.path.join(WORKDIR_ORIG, "DATA", "GDAL_OUTPUT_ORTHO")
os.makedirs(OUTDIR, exist_ok=True)

OUTDIR_DOCKER = "/workspace/DATA/GDAL_OUTPUT_ORTHO"

# =====================================================
# Build GDAL docker
# =====================================================
notebook_dir = os.path.dirname(os.getcwd())
dockerfile_dir = os.path.abspath(os.path.join(
    notebook_dir,
    "src",
    "gdal-latest"
))

print("\n=== BUILDING GDAL LATEST CONTAINER ===\n")
cmd_build = [
    "docker", "build",
    "--platform=linux/amd64",
    "-t", "gdal-latest",
    dockerfile_dir
]

print("Command:", " ".join(cmd_build), "\n")
subprocess.run(cmd_build, check=True)
print("GDAL image built successfully.\n")

# =====================================================
# Generate gdal_ortho.sh
# =====================================================
gdal_script_path = os.path.join(WORKDIR_ORIG, "src", "gdal_ortho.sh")
os.makedirs(os.path.dirname(gdal_script_path), exist_ok=True)

vrt_array = " ".join([f'"{v}"' for v in vrt_names])

workspace_input = "/workspace_tmp" if use_tmp else "/workspace"
with open(gdal_script_path, "w") as f:
    f.write(f"""#!/bin/bash
set +e

cd {workspace_input}/DATA/{product}

OUT_ORTHO="/workspace/DATA/GDAL_OUTPUT_ORTHO"
OUT_MOSAIC="/workspace/DATA/GDAL_OUTPUT_MOSAIC"

mkdir -p "$OUT_ORTHO"
mkdir -p "$OUT_MOSAIC"

XML="{xml_docker}"

echo "=== ORTHORECTIFICATION ==="

for VRT in {vrt_array}; do
    BASENAME="$VRT"
    OUT="$OUT_ORTHO/${{BASENAME}}_ortho.tif"

    echo "----------------------------------------"
    echo "Processing VRT: $VRT"
    echo "----------------------------------------"

    rm -f "$OUT"

    gdalwarp \\
        SENTINEL2_L1B_WITH_GEOLOC:./$XML:$VRT \\
        "$OUT" \\
        -t_srs EPSG:{UTM_EPSG} \\
        -te {LOCATION["ul_x"]} {LOCATION["lr_y"]} {LOCATION["lr_x"]} {LOCATION["ul_y"]} \\
        -r bilinear \\
        -co COMPRESS=LZW \\
        -co TILED=YES \\
        -overwrite

    echo ""
done

echo ""
echo "=== MOSAIC GENERATION ==="
echo ""

for BAND in {" ".join(ORTHO_SETTINGS["keep_bands"])}; do
    echo "----------------------------------------"
    echo "Creating mosaic for band: $BAND"
    echo "----------------------------------------"

    INPUT_FILES=($(ls $OUT_ORTHO/*_${{BAND}}_ortho.tif 2>/dev/null))

    if [ ${{#INPUT_FILES[@]}} -eq 0 ]; then
        echo "No ortho images found for band $BAND"
        continue
    fi

    OUTPUT="$OUT_MOSAIC/ORTHO_mosaic_${{BAND}}.tif"

    gdalwarp \\
        "${{INPUT_FILES[@]}}" \\
        "$OUTPUT" \\
        -r bilinear \\
        -dstnodata 0 \\
        -srcnodata 0 \\
        -multi \\
        -wm 2048 \\
        -overwrite \\
        -co COMPRESS=LZW \\
        -co TILED=YES \\
        -ot UInt16

    echo " Mosaic written -> $OUTPUT"
    echo ""
done

echo "=== GDAL processing complete ==="
""")

os.chmod(gdal_script_path, 0o755)
print("Generated:", gdal_script_path)

# =====================================================
# Run GDAL docker
# =====================================================
print("\n=== RUNNING GDAL PROCESSING ===\n")

cmd_run = [
    "docker", "run",
    "--rm",
    "-v", f"{WORKDIR_ORIG}:/workspace",
]

if use_tmp:
    cmd_run += ["-v", f"{WORKDIR_INPUT}:/workspace_tmp"]

cmd_run += [
    "gdal-latest",
    "/workspace/src/gdal_ortho.sh"
]

print("Command:", " ".join(cmd_run), "\n")
subprocess.run(cmd_run, check=True)
print("\nGDAL ortho + mosaic complete.\n")

# =====================================================
# Cleanup docker image
# =====================================================
print("Removing gdal-latest image...\n")
subprocess.run(["docker", "rmi", "-f", "gdal-latest"], check=True)
print("GDAL image removed.\n")


# =====================================================
# Cleanup TMP files
# =====================================================

if DELETE_TMP_FILES and use_tmp:
    print("Deleting WORKDIR_TMP:", WORKDIR_INPUT)
    if os.path.exists(WORKDIR_INPUT):
        shutil.rmtree(WORKDIR_INPUT)
    print("WORKDIR_TMP deleted.\n")
